<a href="https://colab.research.google.com/github/sansernsam/ChatGPT-Line-Bot/blob/main/Embedding_Search_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Install Steamship

Run the cell below to install the Steamship python library.

In [1]:
%pip install steamship --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling 

# Step 3 - Set your Steamship API Key

1. Visit https://steamship.com/account/api
2. Run the cell below set your key


In [ ]:
from getpass import getpass
STEAMSHIP_API_KEY = getpass('6082689780:AAFaIl0KSoLjOhyabC7frKIjHHltpr93iOw')

# Step 3 - Write your Steamship Package

You will be glad you used this package wrapper -- even if you are just 
experimenting! It enables you to deploy your code to Steamship as a hosted,
managed API with a single command!

In [ ]:
from steamship import check_environment, RuntimeEnvironments, Steamship, Tag
from steamship.invocable import get, post, PackageService
from typing import Union, List

EMBEDDER = {
  "plugin_handle": "openai-embedder",
  "fetch_if_exists": True,
  "config": {
    "model": "text-embedding-ada-002",
    "dimensionality": 1536,
  }
}

class MyPackage(PackageService):
  """This Steamship package can store and process data with AI in the cloud.

  When you run `ship deploy` from the command line, it will deploy to an
  auto-managed, multi-user API.

  Our hosting platform supports:
  - GPT-3 and GPT-4
  - Whisper
  - DALL-E
  - pix2pix
  - Embedding Search
  - LangChain
  - Zapier
  - Telegram bots
  - .. and more

  Let us know what you build in our Discord! We would love to help and promote
  your amazing creations to the community.

  https://steamship.com/discord
  """

  def __init__(self, *args, **kwargs):
    """This runs every time your package is accessed when deployed."""

    super().__init__(*args, **kwargs)

    # This will create a persistent vector index named "my-embedding-index"
    # Using the "openai-embedder" plugin for embeddings.
    self.index = self.client.use_plugin(
      "embedding-index",
      "my-embedding-index",
      config={ "embedder": EMBEDDER },
      fetch_if_exists=True,
    )

  # When you run `ship deploy`, this annotation tells Steamship
  # to create an API endpoint called `/insert` request path.
  # See README.md for more information about deployment.
  @post("insert")
  def insert(self, items: Union[Tag, List[Tag]]) -> bool:
    # The insert command can accept either a Tag or a list of Tags.
    # Passing in JSON or a dict over HTTP is fine since we'll parse it below.
    # Tag has shape {kind: str, name: str, text: str, value: dict}
    # The `text` of the tag is what will be embedded.
    if type(items) != list:
      items = [items]

    items = [Tag.parse_obj(item) for item in items]
    self.index.insert(items)
    return True

  # When you run `ship deploy`, this annotation tells Steamship
  # to create an API endpoint called `/query` request path.
  # See README.md for more information about deployment.
  @post("search")
  def search(self, query: str = "", k: int = 5) -> List[Tag]:
    """Return the `k` closest items in the embedding index."""
    search_results = self.index.search(query, k=k)
    search_results.wait()
    items = search_results.output.items
    return [item.dict() for item in items]


# Step 4 - Test your Steamship Package



In [ ]:
NZ = {
  "text":
  "New Zealand's South Island brims with majestic landscapes at every turn, from dramatic mountains to spectacular fjords. Here, you can admire the mountains of Fiordland National Park, a UNESCO World Heritage Site, from hiking trails or a boat on Milford Sound.",
  "value": {
    "place": "South Island, New Zealand"
  }
}

FR = {
  "text":
  "The magnetic City of Light draws visitors from around the globe who come to see iconic attractions like the Eiffel Tower, the Louvre and the Arc de Triomphe. But what travelers really fall in love with are the city's quaint cafes, vibrant markets, trendy shopping districts and unmistakable je ne sais quoi",
  "value": {
    "place": "Paris, France"
  }
}

ITEMS = [NZ, FR]

In [ ]:
# First we need to create a new steamship client.
from steamship import Steamship
from uuid import uuid4

# Create a Steamship client in a fresh workspace (so data is isolated)
client = Steamship(workspace=f"temp-{uuid4()}", api_key=STEAMSHIP_API_KEY)

# Now we create an instance of your package above
api = MyPackage(client)

# Let's add some information.
for item in ITEMS:
  print(f"- {item['value']['place']}")
  api.insert(item)

- South Island, New Zealand
- Paris, France


In [ ]:
# Now we'll try using the api we just built!

results = api.search(query="Where should I go to see nice architecture?")

for result in results:
  score = result["score"]
  tag = result["tag"]
  place = tag["value"]["place"]
  print(f"{place} (score: {score})")


Paris, France (score: 0.7952640431113398)
South Island, New Zealand (score: 0.7830265878854157)


# Step 5 - Deploy your steamship package!

Are you ready? This is going to be awesome.

In [ ]:
# First we'll write a requirements.txt file. 
# This needs to contain all of the python dependences.

REQUIREMENTS = """
steamship
"""

with open('requirements.txt', 'w') as requirements:
    requirements.write(REQUIREMENTS)

In [ ]:
# Next we'll need to create a steamship.json file

STEAMSHIP_JSON = """{
	"type": "package",
	"handle": "",
	"version": "",
	"description": "",
	"author": "",
	"entrypoint": "api.handler",
	"public": true,
	"build_config": {
		"ignore": [
			"tests",
      "sample_data",
			"examples"
		]
	},
	"configTemplate": {},
	"steamshipRegistry": {
		"tagline": "My first Steamship Package!",
		"tagline2": "",
		"usefulFor": null,
		"videoURL": null,
		"githubURL": null,
		"demoURL": null,
		"jupyterURL": null,
		"authorGithub": null,
		"authorName": null,
		"authorEmail": null,
		"authorTwitter": null,
		"authorURL": null,
		"tags": [
			"Collab"
		]
	}
}"""

with open('steamship.json', 'w') as steamship_json:
    steamship_json.write(STEAMSHIP_JSON)

In [ ]:
# Now we'll write our api.py to disk
#
# 🚢🚢🚢🚢🚢 IMPORTANT IMPORTANT IMPORTANT 🚢🚢🚢🚢🚢
# 🚢🚢                                         🚢🚢    
# 🚢🚢 This step needs you to do something in  🚢🚢
# 🚢🚢 a cell above! Otherwise you won't be    🚢🚢
# 🚢🚢 deploying the latest version of your    🚢🚢
# 🚢🚢 api!                                    🚢🚢
# 🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢
#
# Go to the cell above with your package definition, and then add the following
# text to the top of it:
#
# %%writefile api.py
#
# Then use Shift-Enter (or the run button) to run the cell.
# You should see a message informing you that you have written api.py to disk.
#
# Your package definition must be completely contained within this cell.
#
#
#
# OK.. now remember to remove the %%writefile api.py if you wish to keep
# editing & running this pacakge in Jupyter.
#
# If you're a Jupyter wizard and can help us remove this odd step,
# please let us know in https://steamship.com/discord! 🙏

In [ ]:
# Finall we'll deploy!

!STEAMSHIP_API_KEY=$STEAMSHIP_API_KEY ship deploy

Steamship PYTHON cli version 2.14.2
Creating / fetching package with handle [test-qa-api]... Done.
Bundling content... Done. 📦
Deploying version 1.0.0 of [test-qa-api]... 🚢
It looks like that version [1.0.0] has already been deployed. Would you like to change the version handle and try again? [Y/n]: Y
What should the new version be? Valid characters are a-z, 0-9, . and - [1.0.0-rc.1]: 
Deploying version 1.0.0-rc.1 of [test-qa-api]... 🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢🚢
Done. 🚢
Deployment was successful. View and share your new package here:

https://steamship.com/packages/test-qa-api



# Step 6 - Test calling your API live

Are you PUMPED!? Because if you got here... your Jupuyter code is now a live web page you can share, with its own API.

Click the link in the cell above to take a look....

And then see below to invoke that live API, remotely, from Python

In [ ]:
MY_PACKAGE_HANDLE = 'test-qa-api' # TODO: Replace with the handle you chose when deploying

In [ ]:
# Let's create a new package instance
#
# But unlike above.. now we'll using our instance in the cloud!

# Let's create a new client, just to keep things separated
client2 = Steamship(workspace=f"temp-{uuid4()}", api_key=STEAMSHIP_API_KEY)

# Now we create an instance of your package in the cloud
# Notice how we're passing it hte string handle you used when you deployed.
cloud_api = client2.use(MY_PACKAGE_HANDLE)

In [ ]:
# Now we'll load data into it
for item in ITEMS:
  print(f"- {item['value']['place']}")
  # Notice how we're no longer calling methods on the python object.
  # Intead, we use instance.invoke(method, **kwargs)
  cloud_api.invoke('insert', items=item)

- South Island, New Zealand
- Paris, France


In [ ]:
# Now let's run the API in the cloud
results = cloud_api.invoke("search", query="Where should I go to see nice architecture?")

for result in results:
  score = result["score"]
  tag = result["tag"]
  place = tag["value"]["place"]
  print(f"{place} (score: {score})")


Paris, France (score: 0.79515)
South Island, New Zealand (score: 0.782933)
